# 対話管理（フレームによる方法）
次に、フレームによる対話管理を行います。


## 事前の設定
- 特になし

In [1]:
# 必要なラブラリを読み込む

import sys, os

ここでは、下記のフレームを実装します。
「制約」の列はそのスロットが埋まる必要があるか否かを表します。つまり、地域とジャンルは必ず指定する必要がありますが、予算は指定されていなくても検索可能とします。

<img src="./img/slot.png" style="width: 400px;"/>

## フレームの定義

まずは、フレームを定義します。スロット名は言語理解結果のスロット名と対応するようにします。

In [2]:
# フレームの定義

# スロット名、制約
frame = [
    ['place', 'mandatory'],
    ['genre', 'mandatory'],
    ['budget', 'optional']
]


## セリフの定義

次に必須の項目に関して、それを尋ねるセリフを定義します。
また、最初と最後の発話も特別に定義しておきます。

In [3]:
# 必須項目を尋ねるセリフ

# 辞書型でKeyをスロット名、Valueをセリフにする
utterances = {
    'place': '地域を指定してください',
    'genre': 'ジャンルを指定してください',
}

# 最初の発話
utterance_start = 'こんにちは。京都レストラン案内です。ご質問をどうぞ。'

# 最後の発話は条件に応じて生成する
def gen_utterance_last(current_frame):

    # Optionalである"budget"が埋まっていれば
    if 'budget' in current_frame:
        system_utterance = '地域は%sで、ジャンルは%s、予算は%sですね。検索します。' % (current_frame['place'], current_frame['genre'], current_frame['budget'])
    # "budget"が埋まっていなければ
    else:
        system_utterance = '地域は%sで、ジャンルは%sですね。検索します。' % (current_frame['place'], current_frame['genre'])
    
    return system_utterance

## 制御部の実装
では、フレームに基づいて対話を制御します。内部変数として、現在のフレームの状態（スロット値）を保持し、入力であるユーザ発話に応じてフレームの状態を更新して、システム発話を出力します。

In [4]:
# フレームの現在の状態を保持
# 辞書型のKeyにスロット名、Valueにスロット値を格納する
current_frame = {}

# フレームで必須の情報がすべて埋まったかどうかを保持する
current_frame_filled = False

# 入力であるユーザ発話に応じて、フレームの状態を更新し、システム発話を出力し
# ただし、ユーザ発話の情報は「意図、スロット名、スロット値」のlistとする
def enter(user_utterance):
    
    global current_frame, current_frame_filled
    
    # １つのユーザ発話に複数のスロットの値が含まれることもある
    for slot_user_utterance in user_utterance:

        # スロット名とスロット値を取得
        input_slot_name = slot_user_utterance['slot_name']
        input_slot_value = slot_user_utterance['slot_value']
        
        # フレームの状態を更新
        current_frame[input_slot_name] = input_slot_value

    system_utterance = ""
    
    # 現在のフレームの状態から制約が"mandatory"で不足しているものを探索
    mandatory_need = False
    for slot in frame:
        
        slot_name = slot[0]
        slot_condition = slot[1]

        if slot_condition == 'mandatory' and slot_name not in current_frame:
            system_utterance = utterances[slot_name]
            mandatory_need = True
            break
    
    # すべての"mandatory"の要素が埋まっていたら終了
    if mandatory_need == False:
        
        # システムの発話を生成
        system_utterance = gen_utterance_last(current_frame)

        current_frame_filled = True
    
    return system_utterance

# 初期状態にリセットする
def reset():
    global current_frame, current_frame_filled
    current_frame = {}
    current_frame_filled = False

## 対話管理のテスト
実装した対話管理をテストしてみます。

In [5]:
# フレーム状態をリセット
reset()

# 初期状態の発話を表示
print("システム発話(0)")
print(utterance_start)

システム発話(0)
こんにちは。京都レストラン案内です。ご質問をどうぞ。


In [6]:
# ユーザ発話(1)を設定
user_utterance = [{'slot_name': 'place', 'slot_value': '京都駅周辺'}]
print('ユーザ発話 (1)')
print(user_utterance)
print()

# 次のシステム発話を表示
print('システム発話 (1)')
print(enter(user_utterance))

ユーザ発話 (1)
[{'slot_name': 'place', 'slot_value': '京都駅周辺'}]

システム発話 (1)
ジャンルを指定してください


In [7]:
# ユーザ発話(2)を設定
user_utterance = [{'slot_name': 'genre', 'slot_value': '和食'}]
print('ユーザ発話 (2)')
print(user_utterance)
print()

# 次のシステム発話を表示
print('システム発話 (2)')
print(enter(user_utterance))

if current_frame_filled:
    print('フレームは全て埋まりました')

ユーザ発話 (2)
[{'slot_name': 'genre', 'slot_value': '和食'}]

システム発話 (2)
地域は京都駅周辺で、ジャンルは和食ですね。検索します。
フレームは全て埋まりました


別のパターンをテストしてみます。今度は"optional"の項目である予算について言及します。

In [8]:
# フレーム状態をリセット
reset()

# 初期状態の発話を表示
print("システム発話 (0)")
print(utterance_start)
print()

# ユーザ発話(1)を設定
# 場所と予算を同時に言及
user_utterance = [{'slot_name': 'place', 'slot_value': '京都駅周辺'}, {'slot_name': 'budget', 'slot_value': '5000円'}]
print('ユーザ発話 (1)')
print(user_utterance)
print()

# 次のシステム発話を表示
print('システム発話 (1)')
print(enter(user_utterance))

システム発話 (0)
こんにちは。京都レストラン案内です。ご質問をどうぞ。

ユーザ発話 (1)
[{'slot_name': 'place', 'slot_value': '京都駅周辺'}, {'slot_name': 'budget', 'slot_value': '5000円'}]

システム発話 (1)
ジャンルを指定してください


In [9]:
# ユーザ発話(2)を設定
user_utterance = [{'slot_name': 'genre', 'slot_value': '和食'}]
print('ユーザ発話 (2)')
print(user_utterance)
print()

# 次のシステム発話を表示
print('システム発話(2)')
print(enter(user_utterance))

if current_frame_filled:
    print('フレームは全て埋まりました')

ユーザ発話 (2)
[{'slot_name': 'genre', 'slot_value': '和食'}]

システム発話(2)
地域は京都駅周辺で、ジャンルは和食、予算は5000円ですね。検索します。
フレームは全て埋まりました
